# Comparing SSN steady state rate to the corresponding linear model steady state rate for a fixed h, then for multiple h values

Result - The rates for the two models come out to be exactly equal since we're comparing the steady state rates

In [ ]:
# Installation
using Revise
using LinearAlgebra,Statistics,StatsBase,Distributions
using Plots,NamedColors ; theme(:default)
using FFTW
using ProgressMeter
using Random
Random.seed!(0)

In [ ]:
# Initializing SSN variables
# one E, one I population

v_rest = -70
tau = [20.0, 10.0]*1E-3   #time constant, [ E, I ]
time_step = 1*1E-3
noise = [0.0 0.0] #ignoring

k_ssn = 0.3
n_ssn = 2
w_ssn = [1.25 -0.65
         1.2 -0.5]

In [ ]:
function rate_powerlaw(v::Float64, v_rest::Real, n::Real, k::Real)
    diff = v - v_rest
    diff = (diff < 0) ? 0 : diff
    return k*(diff^n)
end

In [ ]:
# Euler Method
function dv_ssn(v::Float64, v_rest::Real, h_i::Float64, noise_i::Float64, tau_i::Float64, w_arr::Vector{Float64}, rate_arr::Vector{Float64})
    num_neurons = size(w_arr, 1)
    dv = h_i - v + v_rest
    for i in 1:num_neurons
        dv += w_arr[i]*rate_arr[i]
    end
    return dv*time_step/tau_i
end

In [ ]:
# SSN Simulation
function simulate_with_v!(h, n, w, k, num_steps, t_arr, v_excite, v_excite_rest, v_inhibit, v_inhibit_rest, rate)
    
    @showprogress 1.0 "Running SSN simulation..." for i in 2:num_steps
        
        rate[i,:] = [rate_powerlaw(v_excite[i-1], v_excite_rest, n, k) , rate_powerlaw(v_inhibit[i-1], v_inhibit_rest, n, k)]
        v_excite[i] = v_excite[i-1] + dv_ssn(v_excite[i-1], v_excite_rest, h[1], noise[1], tau[1], [w[1,1], w[1,2]], rate[i,:])
        v_inhibit[i] = v_inhibit[i-1] + dv_ssn(v_inhibit[i-1], v_inhibit_rest, h[2], noise[2], tau[2], [w[2,1], w[2,2]], rate[i,:])
        t_arr[i] = t_arr[i-1] + time_step
    end
    
end

In [ ]:
# Using results from the SSN simulation to initialize a linear model
function create_parameters_for_approximation!(w_linear, h_linear, w_ssn, k_ssn, h_ssn, rate_ssn_stable::Vector{Float64})
    
    w_linear[1,1] = 3*w_ssn[1,1]*sqrt(k_ssn*rate_ssn_stable[1]) + sqrt(k_ssn/rate_ssn_stable[1])*(h_ssn[1]+w_ssn[1,2]*rate_ssn_stable[2])
    w_linear[1,2] = 2*w_ssn[1,2]*sqrt(k_ssn*rate_ssn_stable[1])
    w_linear[2,1] = 2*w_ssn[2,1]*sqrt(k_ssn*rate_ssn_stable[2])
    w_linear[2,2] = 3*w_ssn[2,2]*sqrt(k_ssn*rate_ssn_stable[2]) + sqrt(k_ssn/rate_ssn_stable[2])*(h_ssn[2]+w_ssn[2,1]*rate_ssn_stable[1])
    
    
    h_linear[1] = (2-w_linear[1,1])*rate_ssn_stable[1] - w_linear[1,2]*rate_ssn_stable[2]
    h_linear[2] = (2-w_linear[2,2])*rate_ssn_stable[2] - w_linear[2,1]*rate_ssn_stable[1]

end

In [ ]:
# Euler Method for the linear rate model
# the SSN network has been approximated using Taylor expansion around the steady state of rate. Hence, working with the rate of the linear system
function dr_linear(r::Real, h_i::Real, tau_i::Real, w_arr::Vector{Float64}, rate_arr::Vector{Float64})
    num_neurons = size(w_arr, 1)
    dr = h_i - 2*r
    for i in 1:num_neurons
        dr += w_arr[i]*rate_arr[i]
    end
    return dr*time_step/tau_i
end

In [ ]:
# Linear rate model simulation
function simulate_linear_with_r!(h, w, tau, num_steps, t_arr, rate)
    
    @showprogress 1.0 "Running linear simulation..." for i in 2:num_steps
        
        rate[i,1] = rate[i-1,1] + dr_linear(rate[i-1,1], h[1], tau[1], [w[1,1], w[1,2]], rate[i-1,:])
        rate[i,2] = rate[i-1,2] + dr_linear(rate[i-1,2], h[2], tau[2], [w[2,1], w[2,2]], rate[i-1,:])
        t_arr[i] = t_arr[i-1] + time_step
        
    end
    
end

In [ ]:
# comparing SSN with linear approximated model for a fixed h
function simulate_for_single_h()
    h = 5.0
    t_max = 1
    v_rest = -70
    num_steps = Int(ceil(t_max/time_step))
    t_arr = zeros(num_steps)
    v_excite = zeros(num_steps)
    v_inhibit = zeros(num_steps)
    v_excite[1] = v_rest
    v_inhibit[1] = v_rest
    rate_ssn = zeros(num_steps, 2)
    w = [1.25 -0.65
         1.2 -0.5]
    simulate_with_v!([h, h], 2, w, 0.3, num_steps, t_arr, v_excite, v_rest, v_inhibit, v_rest, rate_ssn)
    w_linear = [0.0 0.0
                0.0 0.0]
    h_linear = zeros(2)
    create_parameters_for_approximation!(w_linear, h_linear, w, 0.3, [h, h], rate_ssn[num_steps,:])
    rate_linear = zeros(num_steps, 2)
    simulate_linear_with_r!(h_linear, w_linear, tau, num_steps, t_arr, rate_linear)
    return t_arr, rate_ssn, rate_linear
end

In [ ]:
t_arr, rate_ssn, rate_linear = simulate_for_single_h()
println([rate_ssn[end,:], rate_linear[end,:]])

In [ ]:
plt = plot( xlabel="time (s)", ylabel="E Rate (Hz)", fmt=:png, legend=:bottomright)
plot!(plt, t_arr , rate_ssn[:,1], label = "ssn", color="dark blue", linestyle=:dash)
plot!(plt, t_arr , rate_linear[:,1], label = "linear", color="blue")

In [ ]:
plt = plot(xlabel="time (s)", ylabel="I Rate (Hz)", fmt=:png, legend=:bottomright)
plot!(plt, t_arr , rate_ssn[:,2], label = "ssn", color="dark red", linestyle=:dash)
plot!(plt, t_arr , rate_linear[:,2], label = "linear", color="red")

In [ ]:
# Extending the above simulation for multiple h values
function simulate_for_multiple_h()
    for i in 2:num_h
        h_ssn[i,:] = h_ssn[i-1,:] .+ step
    end
    
    for i in 1:num_h
        t_max = 1
        num_steps = Int(ceil(t_max/time_step))
        t_arr = zeros(num_steps)
        v_excite_ssn = zeros(num_steps)
        v_inhibit_ssn = zeros(num_steps)
        rate_ssn = zeros(num_steps, 2)
        simulate_with_v!(h_ssn[i,:], n_ssn, w_ssn, k_ssn, num_steps, t_arr, v_excite_ssn, v_rest, v_inhibit_ssn, v_rest, rate_ssn)
        rates_ssn[i,:] = rate_ssn[num_steps,:]
        
        w_linear = [0.0 0.0
                    0.0 0.0]
        h_linear = zeros(2)
        create_parameters_for_approximation!(w_linear, h_linear, w_ssn, k_ssn, h_ssn, rate_ssn[num_steps,:])
        rate_linear = zeros(num_steps, 2)
        simulate_linear_with_r!(h_linear, w_linear, tau, num_steps, t_arr, rate_linear)
        rates_linear[i,:] = rate_linear[num_steps,:]
    end
end

In [ ]:
h_max = 20.0
h_min = 1.0
step = 1
num_h = Int(ceil((h_max-h_min+1)/step))
h_ssn = zeros(num_h,2)
h_ssn[1,:] = [h_min, h_min]
rates_ssn = zeros(num_h,2)
rates_linear = zeros(num_h,2)
simulate_for_multiple_h()

In [ ]:
plt = plot( xlabel="h (mV)", ylabel="E Rate (Hz)", fmt=:png, legend=:bottomright)
plot!(plt, h_ssn[:,1] , rates_ssn[:,1], label = "ssn", color="dark blue", linestyle=:dash)
plot!(plt, h_ssn[:,1] , rates_linear[:,1], label = "linear", color="blue")

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="I Rate (Hz)", fmt=:png, legend=:bottomright)
plot!(plt, h_ssn[:,1] , rates_ssn[:,2], label = "ssn", color="dark red", linestyle=:dash)
plot!(plt, h_ssn[:,1] , rates_linear[:,2], label = "linear", color="red")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 1:num_h
    y_e[i] = abs(rates_linear[i,1] - rates_ssn[i,1])
    y_i[i] = abs(rates_linear[i,2] - rates_ssn[i,2])
end
plt = plot(xlabel = "h (mV)", ylabel="absolute error", fmt = :png, legend=:topright)
plot!(plt, h_ssn[:,1], y_e, ylims=(-0.0001,0.0001), label="rateE", color="blue")
plot!(plt, h_ssn[:,1], y_i, label="rateI", color="red")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 1:num_h
    y_e[i] = 2*abs(rates_linear[i,1] - rates_ssn[i,1])/(rates_linear[i,1] + rates_ssn[i,1])
    y_i[i] = 2*abs(rates_linear[i,2] - rates_ssn[i,2])/(rates_linear[i,2] + rates_ssn[i,1])
end
plt = plot(xlabel = "h (mV)", ylabel="relative error", fmt = :png, legend=:topright)
plot!(plt, h_ssn[:,1], y_e, ylims=(-0.0001,0.0001), label="rateE", color="blue")
plot!(plt, h_ssn[:,1], y_i, label="rateI", color="red")